## Constraint-based structual learning for Bayesian Network
* This is an **Interactive Version**, which unpacks functions for interactive use. For a compact version, please refer to **main.py** or **constraintBN.ipynb**
* The algorithm are implemented based on [Scutari](https://arxiv.org/pdf/1406.7648.pdf)
* The algorithm will run several iterations. Each time, it will go through four main stages: 
    * sampling & preprocessing
    * finding Markov Blankets
    * determining Neighbors
    * learning arc directions
* Attributes and Edges will be returned.
* For interactive purpose, this file will go through one iteration step by step, and it will perform given iterations at the final stage to show the result.

### Load Data & Specifying Parameters
* The data needs to be .csv files and we need to replace all the "," within the cell before processing
* required **filename** and **location** of the dataset, optional parameters are **sample_size**, number of iterations (**iteration**), and **alpha** for independence tests.

In [1]:
import numpy as np
from scipy import stats
from copy import copy
import math
import csv

filename = '500_Cities__Local_Data_for_Better_Health__2017_release.csv'
location = '../datasets/'
sample_size = 100
iteration = 5
alpha = 0.05

### Sampling & Preprocessing
* **reformat**: it uniformly randomly selected [sample_size] records from the dataset and print out the attributes names and their indexes
* **replace_str**: the records of a given dataset will be transformed into numbers for further computing. For example, if a column has [a,b,b,c,d,c], it will become [0,1,1,2,3,2]

In [2]:
'''
from utility.py
'''
def replace_str(data, return_dic = False):
    i = 0
    value_dic = {}
    for col in range(len(data[0])):
        unique = {}
        index = 0
        t = 0
        for row in data:
            if row[col] not in unique.keys():
                unique[row[col]] = index
                row[col] = index
                index+=1
            else:
                row[col] = unique[row[col]]
        value_dic[col] = unique
    if return_dic:
        return data, value_dic
    else:
        return data

def reformat(path, clean_path = "", size = 1000):
    with open(path) as csvfile:
        raw = csvfile.readlines()
        fieldnames = raw[0].strip('\n').split(",")
    raw = raw[1:]
    sample = np.random.choice(len(raw), size)
    sample = sample.tolist()
    split_raw = []
    for i in sample:
        row = raw[i].split(",")
        split_raw.append(row)
    numeric_raw = replace_str(split_raw)
    return numeric_raw, fieldnames

### Finding Markov Blankets
* To find the markov blankets, I mainly used Grow and Shrink Algorithm from [Margaritis's Thesis](https://www.cs.cmu.edu/~dmarg/Papers/PhD-Thesis-Margaritis.pdf).
    * Grow Phase: 
      While $\exists(Y)\in U -{X}$ such that $Y not\perp X|S $ do $S\gets S\cup{Y}$.
    * Shrink Phase:
      While $\exists(Y)\in S $ such that $Y \perp X|S-{Y} $ do $S\gets S-{Y}$.
* After finding all the MB, I will perform a symmetric check (when x belongs to y's blanket, if y belongs to x's blanket) and drop out those that do not hold to reduce false postives.

In [3]:
from utility import *
'''
from learnAlg.py: learn Markov Blanket Using GS
'''
def gs(data, alpha):
    # number of attributes
    n_attr = data.shape[1]
    # number of records
    n_rec = data.shape[0]
    col_index = range(n_attr)
    # init empty blanket container for each attri
    blanket = dict([(i,[]) for i in range(n_attr)])
    for X in col_index:
        # step 1: init blanket for attri
        S = []
        # step2: GROWING phase
        for Y in col_index:
            # exists Y not belonging to X nor S
            if X != Y and Y not in S:
                columns = (X,Y) + tuple(S)
                if not are_independent(data[:,columns]):
                    S.append(Y)
        # step3: SHRINKING phase
        for Y in S:
            # test if Y == X 
            if X != Y:
                new_S = copy(S)
                new_S.remove(Y)
                columns = (X,Y) + tuple(new_S)
                # Y indep of X given S - Y, S = S - Y
                if are_independent(data[:,columns]):
                    S = new_S
        # save to blanket
        blanket[X] = S
    return blanket

#### Independence Test
* In Grow and Shrink Algorithm, we used **are_independent** and **alpha** to test the independence or conditional independence of X and Y.
* The independence tests are developed based on Daphne's book in Chapter 18.2.2, page 789-790 -- Using chi-sqaure to calculate the deviance and p-value, then comparing the p-value with a threshold of alpha (default: 0.05).
* Notes:
    * Here I used 1e-7 to avoid division by zero.
    * If there are more than 3 columns passed, I will concatenate all the columns after the second column to be a single Z. In this way, the dimension of computing is always <= 3. 

In [4]:
'''
from utility.py:
'''
def are_independent(data, alpha = 0.05):
    pval = indep_test(data)
    if pval < alpha:
        return True
    else:
        return False

'''
Independent tests:
@param test: perform chi-square test
For data = [X,Y]
- calculate joint prob
- calculate marginal prob
- cross product of marginal X and marginal Y
- calculate chi2 statistics
'''
def indep_test(data, test=True):
    bins = unique_bins(data)
    n_row = data.shape[0]
    if len(bins) == 2:
        # PAGE 788-789
        # frequency counts
        hist,_ = np.histogramdd(data, bins=bins[0:2]) 
        # joint probability distribution over X,Y,(Z)
        Pxy = hist / data.shape[0] 
        # marginal: axis 0: combine rows/across X; axis 1: combine cols/across Y
        Px = np.sum(Pxy, axis = 1) # P(X,Z)
        Py = np.sum(Pxy, axis = 0) # P(Y,Z) 
        # avoid division by zero
        Px += 1e-7
        Py += 1e-7
        # deviance using chi-square
        chi = 0
        for i in range(bins[0]):
            for j in range(bins[1]):
                chi += n_row * math.pow(Pxy[i][j] - Px[i] * Py[j], 2) / Px[i] * Py[j]
        dof = (bins[0] - 1) * (bins[1] - 1)
        p_val = 2*stats.chi2.pdf(chi, dof) # 2* for one tail
        return round(p_val,4)
    
    else:
        # PAGE 790, condition on Z
        # CHECK FOR > 3 COLUMNS -> concatenate Z into one column
        if len(bins) > 3:
            data = data.astype('str')
            ncols = len(bins)
            for i in range(len(data)):
                data[i,2] = ''.join(data[i,2:ncols])
            data = data.astype('int')[:,0:3]

        bins = unique_bins(data)
        hist,_ = np.histogramdd(data, bins=bins)

        # joint probability distribution over X,Y,Z
        Pxyz = hist / n_row
        Pz = np.sum(Pxyz, axis = (0,1)) # P(Z)
        Pxz = np.sum(Pxyz, axis = 1) # P(X,Z)
        Pyz = np.sum(Pxyz, axis = 0) # P(Y,Z)   

        Pxy_z = Pxyz / (Pz+1e-7) # P(X,Y | Z) = P(X,Y,Z) / P(Z)
        Px_z = Pxz / (Pz+1e-7) # P(X | Z) = P(X,Z) / P(Z)   
        Py_z = Pyz / (Pz+1e-7) # P(Y | Z) = P(Y,Z) / P(Z)

        Px_y_z = np.empty((Pxy_z.shape)) # P(X|Z)P(Y|Z)
        
        # avoid division by zero
        Pz += 1e-7
        
        # (M[x,y,z] - M*P(z)P(x|z)P(y|z))^2 / M * P(z)P(x|z)P(y|z)
        chi = 0
        for i in range(bins[0]):
            for j in range(bins[1]):
                for k in range(bins[2]):
                    Px_y_z[i][j][k] = Px_z[i][k]*Py_z[j][k] + 1e-7
                    chi += n_row * math.pow((Pxyz[i][j][k] - Pz[k] * Px_y_z[i][j][k]), 2) / (Pz[k] * Px_y_z[i][j][k])
        dof = (bins[0] - 1) * (bins[1] - 1) * bins[2]
        p_val = 2*stats.chi2.pdf(chi, dof) # 2* for one tail
        
        return round(p_val,4)
        

#### Symmetric Check
* The step is used to reduce false positives.

In [5]:
'''
from learnAlg.py: check symmetric for mb or nb
'''
def check_symmetric(mb):
    new_mb = dict(mb)
    attr = mb.keys()
    for x in attr:
        for i in mb[x]:
            if x not in mb[i]:
                new_mb[x].remove(i)
    return new_mb

### Determining Neighbors
* For each pair of attribute X and Y, where X is not the same as Y, search for a set (including empty set) on which X and Y are independent. If no such set exists, place an undirected arc between X and Y.
* In this step, I used MB to reduce the search space. Specifically:
    * if X in Y's MB:
        * search all the subsets of MB(Y). once found a subset seperating X and Y -> they are not neighbors
        * if no such set found, test independence of X and Y without conditions
        * if still not independent, add Y to X's neighbor
    * if X not in Y's MB:
        * given MB(Y), X and Y must be independent -> they are not neighbors
* check symmetric again

In [6]:
'''
from learnAlg.py: learn neighbours
'''
def learnNb(data, mb, alpha):
    nb = {}
    # N(x) is subset of B(x)
    for x in range(data.shape[1]):
        nb[x] = []
        for y in range(data.shape[1]):
            if x in mb[y]:
                space = copy(mb[y]).remove(x)
                noset = True
                if space != None:
                    subset = find_subset(space)
                    for s in subset.values():
                        columns = (x,y,s)
                        if are_independent(data[:,columns]):
                            noset = False
                            break
                # test empty s
                columns = (x,y)
                if are_independent(data[:,columns]):
                    noset = False
                if noset:
                    nb[x].append(y)
                    # place an undirected edge beteewn x and y
                    #print "{} and {} has an edge".format(x, y)
    return check_symmetric(nb)

### Learning Arc Directions
* 1) Learn V-structure. For each non-adjacent X,Y with a common neighbor S, check if given Z, X and Y are independent. If not, create a v structure with {X -> S <- Y}
* 2) After learning v-structures, recursively check two rules:
    * If X - Y and there is a directed path from X to Y, then change X-Y to X->Y
    * If X, Y are not adjacent, check if an S exists such that X -> S, S - Y, then change S - Y to S -> Y.
* also referenced Chap 3.3-3.4 in Daphne's book.

In [7]:
'''
from learnAlg.py: learn arc directions
'''
def learnDir(data, nb, alpha):
    leftToRight = {}
    # find V-structure
    for x in nb.keys():
        leftToRight[x] = []
        for y in range(x+1, data.shape[1]):
            # find non-adjacent x,y
            if y in nb[x]:
                continue
            # find their common neighbor
            commonNb = list(set(nb[x]).intersection(nb[y]))
            for s in commonNb:
                # check if x and y are independent given common neighbour belongs
                columns = (x,y,s)
                if not are_independent(data[:,columns]):
                    if s not in leftToRight[x]:
                        leftToRight[x].append(s)
                    if y not in leftToRight.keys():
                        leftToRight[y] = []
                    if s not in leftToRight[y]:
                        leftToRight[y].append(s)
                    #print "{} -> {} <- {}".format(x, s, y)
    # recursively applying two rules util converge
    last = {}
    while last != leftToRight:
        last = copy(leftToRight)
        for x in nb.keys():
            for y in nb.keys():
                # case1: adjacent
                if y in nb[x]:
                    # find undirected edges
                    if y in leftToRight[x] or x in leftToRight[y]:
                        continue
                    # if existing a directed path from x to y, set x -> y
                    if hasDirectedPath(x,y,leftToRight):
                        if y not in leftToRight[x]:
                            leftToRight[x].append(y)
                        #print "{} -> {}".format(x, y)

                # case2: non-adjacent
                # if existing s that x -> s and s - y. set s -> y
                else:
                    for s in leftToRight[x]:
                        if s in nb[y]:
                            # not s <- y
                            if y not in leftToRight[s] and s not in leftToRight[y]:
                                leftToRight[s].append(y)
                            #print "{} -> {}".format(s, y)
    return leftToRight

'''
recursive call to find a directed path from x to y
'''
def hasDirectedPath(x, y, leftToRight):
    if leftToRight[x] == None:
        return False
    if y in leftToRight[x]:
        return True
    else:
        for i in leftToRight[x]:
            if hasDirectedPath(i, y, leftToRight):
                return True

### Iterations and Count Occurences of Each Edge
* After all iterations are done, **edges** are returned in a form like {left node:{right node: # occurences in all iterations}} or you could use **printEdge** to print the edges exceeding a specified **threshold**.
* Example for threshold: if edge x -> y appeared 6 times in 10 iterations, with a threshold of 0.5, since 6 > 10*0.5, the edge will be displayed by the printEdge function.

In [13]:
def constraintBN(filename, location, sample_size = 100, iteration = 1, alpha = 0.5):
    # left -> right
    edge = {}
    print "sample_size: {}, iterations: {}, alpha: {}".format(sample_size, iteration, alpha)
    for i in range(iteration):
        print "iteration {}".format(i)
        path = location + filename
        #clean_path = location + filename.split(".")[0] + "-num.csv"
        # Reformat data and replace string, size = sample_size
        data, field = reformat(path, size = sample_size)
        if i == 0:
            printAttr(field)
        data = np.array(data, np.int32)
        #data = np.genfromtxt(clean_path, dtype='int32', delimiter=',')
        # 1. Find Markov Blankets
        mb = gs(data, alpha = alpha)
        mb = check_symmetric(mb)
        # 2. learning neighbors
        nb = learnNb(data, mb, alpha = alpha)
        nb = check_symmetric(nb)
        # 3. learning directions
        arc = learnDir(data, nb, alpha = alpha)
        # calculate occurences
        for left in arc.keys():
            right = arc[left]
            if left not in edge.keys():
                edge[left] = {}
            for r in right:
                if r not in edge[left].keys():
                    edge[left][r] = 1
                else:
                    edge[left][r] += 1
    printEdge(edge, itr = iteration)
    return edge

def printEdge(edge, itr, threshold = 0.8):
    for e in edge:
        right = edge[e]
        for r in right:
            if edge[e][r] > threshold*itr:
                print "{} -> {} ({})".format(e, r, edge[e][r])

In [14]:
edges = constraintBN(filename, location, sample_size = sample_size, iteration = iteration, alpha = alpha)

sample_size: 100, iterations: 5, alpha: 0.5
iteration 0
Attributes - Index
Year - 0
StateAbbr - 1
StateDesc - 2
CityName - 3
GeographicLevel - 4
DataSource - 5
Category - 6
UniqueID - 7
Measure - 8
Data_Value_Unit - 9
DataValueTypeID - 10
Data_Value_Type - 11
Data_Value - 12
Low_Confidence_Limit - 13
High_Confidence_Limit - 14
Data_Value_Footnote_Symbol - 15
Data_Value_Footnote - 16
PopulationCount - 17
GeoLocation - 18
CategoryID - 19
MeasureId - 20
CityFIPS - 21
TractFIPS - 22
 - 23_Question_Text
iteration 1
iteration 2
iteration 3
iteration 4
0 -> 5 (5)
0 -> 9 (5)
4 -> 5 (5)
4 -> 9 (5)
5 -> 13 (5)
5 -> 14 (5)
5 -> 17 (5)
5 -> 18 (5)
5 -> 19 (5)
5 -> 20 (5)
5 -> 21 (5)
5 -> 22 (5)
5 -> 23 (5)
7 -> 5 (5)
7 -> 9 (5)
9 -> 13 (5)
9 -> 14 (5)
9 -> 17 (5)
9 -> 18 (5)
9 -> 19 (5)
9 -> 20 (5)
9 -> 21 (5)
9 -> 22 (5)
9 -> 23 (5)


### Limitations
#### Limited Representation
* BN can only be used to represent causal relationship and fail to represnet correlations.
* Fail to represent distributions like {A $\perp$ C | B, D} and {B $\perp$ D | A, C}

#### Complexity